In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from netCDF4 import Dataset
import os
import pandas as pd
import datetime as dt
from h5py import File

In [ ]:
def read_msat_dark_file(ID=None,bad_pixel_mask=None):
    try:
        with Dataset(filename,'r') as fid:
            dark = np.nanmean(fid['Frame/PixelData'][:],0).data
            dark[np.nanmean(fid['Frame/PixelData'][:],0).mask] = np.nan
            dark[bad_pixel_mask > 0] = np.nan
            dark_std = np.nanstd(fid['Frame/PixelData'][:],0).data
            dark_std[np.nanmean(fid['Frame/PixelData'][:],0).mask] = np.nan
            dark[bad_pixel_mask > 0] = np.nan
            time_stamp = fid['Frame/TimeStamp'][0].data.astype(int)//1e6
    except:
        print(f'{ifl} returned exception')
    return_dict = {'ID':ID,'dark':dark,'dark_std':dark_std,'time':time_stamp}
    return return_dict

def run_function_in_parallel(fun,args_list):
    """
    Description:        Generic function that runs any function over a set of CPUs -
                        note that this uses concurrent.futures.PoolProcessExecutor due
                        to multithreading issues with rpy2
    Inputs:
        fun             Python function
        args_list       List of dictionaries with keys set to match the input variable
                        argument names for the function fun. Note: each dictionary must have
                        a key called ID with unique value to be identifiable in the results dictionary.
    Returns:
        results_dict    Concatenated results of the function from collection of args_list
                        indexed by the ID key in the args_list list of dictionaries. Each entry
                        has a "result" entry that is the return value of the function, a "warnings" 
                        entry, and if the function does not conclude successfully, an error entry 
                        consisting of the Exception.
    """
    result_dict = {}
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit tasks to be executed in parallel with named arguments
        future_to_args = {executor.submit(fun, **args): args for args in args_list}

        # Gather results as they complete
        for future in concurrent.futures.as_completed(future_to_args):
            args_future = future_to_args[future]
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always")
                    result = future.result()
                    result_dict[args_future['ID']] = {
                            "result": result,
                            "warnings": [str(warn.message) for warn in w],
                            "error": None
                            }
                print(f"Function with arg ID {args_future['ID']} finished successfully!")
            except Exception as exc:
                print(f"Function with arg ID {args_future['ID']} generated an exception: {exc}")
                result_dict[args_future['ID']] = {
                            "result": None,
                            "warnings": [],
                            "error": f"Generated an exception: {exc}"
                            }
    return result_dict

In [2]:
#os.chdir('/data1/crowell/methanesat/darks/')
ch4_files = sorted(list(Path('./').rglob('*CH4*.nc')))
o2_files = sorted(list(Path('./').rglob('*O2*.nc')))


In [3]:
ch4_bp = Dataset('../BadPixelMap_CH4_20250422_RTSplusHot.nc','r')['BadPixelMap'][:] 
o2_bp = Dataset('../BadPixelMap_O2_20250422_RTSplusHot.nc','r')['BadPixelMap'][:] 

In [4]:
ch4_dark = np.zeros((len(ch4_files),2048,2048))
ch4_dark_std = np.zeros((len(ch4_files),2048,2048))
ch4_nframes = np.zeros(len(ch4_files)).astype(int)
dk_ts = np.zeros(len(ch4_files)).astype(np.int64)

for ifl in range(len(ch4_files)):
    print(ifl)
    try:
        with Dataset(ch4_files[ifl],'r') as fid_ch4:
            ch4_dark[ifl] = np.nanmean(fid_ch4['Frame/PixelData'][:],0).data
            ch4_dark[ifl][np.nanmean(fid_ch4['Frame/PixelData'][:],0).mask] = np.nan
            ch4_dark[ifl][ch4_bp > 0] = np.nan
            ch4_dark_std[ifl] = np.nanstd(fid_ch4['Frame/PixelData'][:],0).data
            ch4_dark_std[ifl][np.nanmean(fid_ch4['Frame/PixelData'][:],0).mask] = np.nan
            ch4_dark[ifl][ch4_bp > 0] = np.nan
            dk_ts[ifl] = fid_ch4['Frame/TimeStamp'][0].data.astype(int)//1e6
    except:
        print(f'{ifl} returned exception')
ch4_dk_dt = np.array([ dt.datetime(1980,1,1) + dt.timedelta(seconds=int(dk_ts[i])) for i in range(len(ch4_files))])




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
ch4 = {}
ch4['dark'] = ch4_dark
ch4['dark_std'] = ch4_dark_std
ch4['sec_since_1970_01_01'] = np.array([ (d-dt.datetime(1970,1,1)).total_seconds() for d in ch4_dk_dt])

with File('ch4_darks.h5','w') as f:
    try:
        for ky in ch4.keys():
            print(ky)
            f.create_dataset(ky,data=ch4[ky][:])
    except:
        print('Oops! Something went wrong!')
    f.close()

dark
dark_std
sec_since_1970_01_01


In [6]:
o2_dark = np.zeros((len(o2_files),2048,2048))
o2_dark_std = np.zeros((len(o2_files),2048,2048))
o2_nframes = np.zeros(len(o2_files)).astype(int)
o2_dk_ts = np.zeros(len(o2_files)).astype(np.int64)
o2_good_file = []
for ifl in range(len(o2_files)):
    print(ifl)
    try:
        with Dataset(o2_files[ifl],'r') as fid:
            o2_dark[ifl] = np.nanmean(fid['Frame/PixelData'][:],0).data
            o2_dark[ifl][np.nanmean(fid['Frame/PixelData'][:],0).mask] = np.nan
            o2_dark[ifl][o2_bp > 0] = np.nan
            o2_dark_std[ifl] = np.nanstd(fid['Frame/PixelData'][:],0).data
            o2_dark_std[ifl][np.nanmean(fid['Frame/PixelData'][:],0).mask] = np.nan
            o2_dark[ifl][o2_bp > 0] = np.nan
            o2_dk_ts[ifl] = fid['Frame/TimeStamp'][0].data.astype(int)//1e6
            o2_good_file.append(True)
    except:
        print(f'{ifl} returned exception')
        o2_good_file.append(False)
        
o2_dk_dt = np.array([ dt.datetime(1980,1,1) + dt.timedelta(seconds=int(o2_dk_ts[i])) for i in range(len(o2_files))])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
o2 = {}
o2['dark'] = o2_dark
o2['dark_std'] = o2_dark_std
#o2['nframes'] = o2_nframes
o2['sec_since_1970_01_01'] = np.array([ (d-dt.datetime(1970,1,1)).total_seconds() for d in o2_dk_dt])
#o2['good_file'] = o2_good_file

with File('o2_darks.h5','w') as f:
    try:
        for ky in o2.keys():
            print(ky)
            f.create_dataset(ky,data=o2[ky][:])
    except:
        print('Oops! Something went wrong!')
    f.close()

dark
dark_std
sec_since_1970_01_01


In [8]:
mean_ch4_dark = np.nanmean(ch4_dark,axis=(1,2))
mean_o2_dark = np.nanmean(o2_dark,axis=(1,2))

In [9]:
ch4_df = pd.DataFrame({'mean_ch4_dark':mean_ch4_dark},index=dk_dt)
ch4_df = ch4_df[ch4_df.index > '2024-01-01']
o2_df = pd.DataFrame({'mean_o2_dark':mean_o2_dark},index=o2_dk_dt)
o2_df = o2_df[o2_df.index > '2024-01-01']

NameError: name 'dk_dt' is not defined

In [ ]:
ch4_df.plot()
plt.gca().set_ylim([1324,1334])
o2_df.plot()
plt.gca().set_ylim([1140,1150])

## Take a look at the first file to get a sense of the variation across frames

In [ ]:
fid_ch4 = Dataset(ch4_files[0],'r')
fid_o2 = Dataset(o2_files[0],'r')
dk2 = fid_ch4['Frame/PixelData'][:]
dk2_std = np.nanstd(fid_ch4['Frame/PixelData'][:],0)
t = fid_ch4['Frame/TimeStamp'][:].data.astype(int)//1e6
for i in range(20):
    dk2.mask[i] = np.logical_or(dk2.mask[i],(ch4_bp > 0))
dk2_std.mask = np.logical_or(dk2_std.mask,(ch4_bp > 0))

In [ ]:
fid_ch4['CollectMetadata']

In [ ]:
fig,axs = plt.subplots(5,4,figsize=(20,20))
for i in range(20):
    ax = axs[i//4,i%4]
    g = ax.pcolormesh(dk2[i],vmin=1400,vmax=1600); plt.colorbar(g,ax=ax)

In [ ]:
fig,axs = plt.subplots(5,4,figsize=(20,20))
for i in range(1,20):
    ax = axs[i//4,i%4]
    msk = np.logical_or(dk2[i].mask,dk2.mean(0).mask)
    df = dk2[i].data.astype(np.float32)-dk2.mean(0).data.astype(np.float32)
    df[msk] = np.nan
    g = ax.pcolormesh(df,vmin=-15,vmax=15); plt.colorbar(g,ax=ax)

In [ ]:
plt.pcolormesh(dk2_std); plt.colorbar()

In [ ]:
plt.plot(dk2[0,700:750,1500:1750].mean(1).T-dk2[1,700:750,1500:1750].mean(1).T)
plt.plot(dk2[0,700:750,1500:1750].mean(1).T-dk2[2,700:750,1500:1750].mean(1).T)
plt.plot(dk2[0,700:750,1500:1750].mean(1).T-dk2[3,700:750,1500:1750].mean(1).T)


In [ ]:
fid_ch4 = Dataset(ch4_files[1],'r')
fid_o2 = Dataset(o2_files[1],'r')
dk2_2 = fid_ch4['Frame/PixelData'][:]
dk2_std_2 = np.nanstd(fid_ch4['Frame/PixelData'][:],0)
for i in range(dk2_2.shape[0]):
    dk2_2.mask[i] = np.logical_or(dk2_2.mask[i],(ch4_bp > 0))
dk2_std_2.mask = np.logical_or(dk2_std_2.mask,(ch4_bp > 0))

In [ ]:
fig,axs = plt.subplots(dk2_2.shape[0]//4+1,4,figsize=(20,20))
for i in range(dk2_2.shape[0]):
    ax = axs[i//4,i%4]
    g = ax.pcolormesh(dk2_2[i],vmin=1400,vmax=1600); plt.colorbar(g,ax=ax)

## At least the first and second dark are pretty similar

In [ ]:
msk = np.logical_or(dk2.mean(0).mask,dk2_2.mean(0).mask)
df = dk2.mean(0).data.astype(np.float32)-dk2_2.mean(0).data.astype(np.float32)
df[msk] = np.nan
plt.pcolormesh(df); plt.colorbar()
plt.figure();
plt.hist(df.flatten())

In [ ]:
np.diff(fid_ch4['Frame/TimeStamp'][:])
fid_ch4['Frame/TimeStamp']

# Time Series of Darks

In [ ]:
plt.pcolormesh(ch4_dark[9]-ch4_dark[0],vmin=-15,vmax=15); plt.colorbar()

In [ ]:
plt.pcolormesh(ch4_dark[9]-ch4_dark[8],vmin=-15,vmax=15); plt.colorbar()

In [ ]:
mean_dark = np.nanmean(ch4_dark,(1,2))[:]
df = pd.DataFrame({'mean_dk':mean_dark},index=dk_dt)

In [ ]:
df = df[df.index > '2024-01-01']

In [ ]:
df.plot()
plt.gca().set_ylim([1300,1350])

In [ ]:
plt.plot(mean_dark)
plt.gca().set_ylim([1325,1335])
#plt.gca().set_xlim([439,479])

In [ ]:
plt.plot(mean_dark)
plt.gca().set_ylim([1325,1335])
plt.gca().set_xlim([439,479])

In [ ]:
mean_dark[0]
plt.hist(ch4_bp.flatten())

In [ ]:
plt.pcolormesh(ch4_dark[441]-ch4_dark[440],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('Change from 440 to 441 (Dip2)');
plt.figure(); plt.pcolormesh(ch4_dark[439]-ch4_dark[438],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('Change from 438 to 439');
plt.figure(); plt.pcolormesh(-(ch4_dark[440]-ch4_dark[441])+(ch4_dark[438]-ch4_dark[439]),vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('438 to 439 change - 440 to 441 change');

In [ ]:
plt.pcolormesh(ch4_dark[443]-ch4_dark[442],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar()
plt.figure(); plt.pcolormesh(ch4_dark[441]-ch4_dark[440],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar()
plt.figure(); plt.pcolormesh((ch4_dark[443]-ch4_dark[442])-(ch4_dark[441]-ch4_dark[440]),cmap=plt.cm.RdBu,vmin=-5,vmax=5); plt.colorbar()

In [ ]:
plt.hist((ch4_dark[443]-ch4_dark[442]).flatten(),bins=np.linspace(-20,20,21))
plt.hist((ch4_dark[441]-ch4_dark[440]).flatten(),bins=np.linspace(-20,20,21))
plt.hist((ch4_dark[443]-ch4_dark[442]).flatten()-(ch4_dark[441]-ch4_dark[440]).flatten(),bins=np.linspace(-20,20,21))

In [ ]:
plt.pcolormesh(ch4_dark[463]-ch4_dark[462],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('Change from 462 to 463 (Dip2)');
plt.figure(); plt.pcolormesh(ch4_dark[461]-ch4_dark[460],vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('Change from 460 to 461');
plt.figure(); plt.pcolormesh((ch4_dark[463]-ch4_dark[462])-(ch4_dark[461]-ch4_dark[460]),vmin=-5,vmax=5,cmap=plt.cm.RdBu); plt.colorbar(); plt.title('460 to 461 change - 462 to 463 change');

In [ ]:
plt.hist((ch4_dark[441]-ch4_dark[440]).flatten(),bins=np.linspace(-20,20,21),histtype='step')
plt.hist((ch4_dark[439]-ch4_dark[438]).flatten(),bins=np.linspace(-20,20,21),histtype='step')
plt.hist((ch4_dark[441]-ch4_dark[440]).flatten()-(ch4_dark[439]-ch4_dark[438]).flatten(),bins=np.linspace(-20,20,21))
plt.legend(['Dip1','Pre-Dip1','Delta1'])
plt.figure()
plt.hist((ch4_dark[463]-ch4_dark[462]).flatten(),bins=np.linspace(-20,20,21),histtype='step')
plt.hist((ch4_dark[461]-ch4_dark[460]).flatten(),bins=np.linspace(-20,20,21),histtype='step')
plt.hist((ch4_dark[463]-ch4_dark[462]).flatten()-(ch4_dark[461]-ch4_dark[460]).flatten(),bins=np.linspace(-20,20,21))
plt.legend(['Dip2','Pre-Dip2','Delta2'])

In [ ]:
!pip install h5py
from h5py import File
f = File('mean_darks.h5','w')
f.create_dataset('dark',data=ch4_dark)
f.close()

In [ ]:
df = pd.read_csv('../telemetry/2025_01_02_21_00_00_science_data_processing_telemetry_report.csv')

In [ ]:
df['T_OP_CH4_FPA_TEMP'][5::6].plot()